In [1]:
#Necessary Imports

In [91]:
import numpy as np
import pandas as pd


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import re
import xgboost as xgb
from sklearn import preprocessing, ensemble

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('train_individual.csv')

In [4]:
#data.isna().sum()

Taking latest data '2016-05-28'

In [5]:
data = data.loc[data['fecha_dato']=='2016-05-28']

Date Processing to extract tenure of a Customer

In [6]:
data['fecha_dato'] = pd.to_datetime(data['fecha_dato'])
data['fecha_alta'] = pd.to_datetime(data['fecha_alta'])

data['Tenure'] = data['fecha_dato']-data['fecha_alta']

Selecting the following columns as *predictors* for Analysis

In [7]:
#selecting columns 
mapping_dict = {
'ind_empleado'  : {-99:0, 'N':1, 'B':2, 'F':3, 'A':4, 'S':5},
'sexo'          : {'V':0, 'H':1, -99:2},
'ind_nuevo'     : {'0':0, '1':1, -99:2},
'indrel'        : {'1':0, '99':1, -99:2},
'indrel_1mes'   : {-99:0, '1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'tiprel_1mes'   : {-99:0, 'I':1, 'A':2, 'P':3, 'R':4, 'N':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'indext'        : {-99:0, 'S':1, 'N':2},
'conyuemp'      : {-99:0, 'S':1, 'N':2},
'indfall'       : {-99:0, 'S':1, 'N':2},
'tipodom'       : {-99:0, '1':1},
'ind_actividad_cliente' : {'0':0, '1':1, -99:2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:2},
'pais_residencia' : {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27},
'canal_entrada' : {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11}
}
cat_cols = list(mapping_dict.keys())
cat_cols.append('Tenure')
cat_cols.append('age')
cat_cols.append('renta')
cat_cols.append('antiguedad')

Selecting the following columns as *targets* for Analysis

In [8]:
target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

Separating Predictors and Targets

In [9]:
trainX = data[cat_cols]

In [10]:
trainY = data[target_cols]

Verifying the shape of trainX and trainY

In [11]:
print(trainX.shape,trainY.shape)

(542596, 19) (542596, 24)


In [12]:
#trainX.isna().sum()

Imputing missing values in the Training set

In [13]:
trainX['indrel_1mes'].fillna(-99,inplace=True)
trainX['renta'].fillna(101850,inplace=True)
trainX['ind_nuevo'].fillna(-99,inplace=True)
trainX['tipodom'].fillna(-99,inplace=True)
trainX['ind_actividad_cliente'].fillna(-99,inplace=True)
trainX['age'].fillna(40,inplace=True)
trainX['indrel'].fillna(-99,inplace=True)

trainX['tiprel_1mes'].fillna('missing',inplace=True)
trainX['ind_empleado'].fillna('missing',inplace=True)
trainX['pais_residencia'].fillna('missing',inplace=True)
trainX['sexo'].fillna('missing',inplace=True)
trainX['tiprel_1mes'].fillna('missing',inplace=True)
trainX['conyuemp'].fillna('missing',inplace=True)
trainX['segmento'].fillna('missing',inplace=True)
trainX['canal_entrada'].fillna('missing',inplace=True)
trainX['indresi'].fillna('missing',inplace=True)
trainX['indext'].fillna('missing',inplace=True)
trainX['indfall'].fillna('missing',inplace=True)

Handling Continuous variables from Objects to Numeric

In [14]:
for item in ['Tenure','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']:
    trainX[item] = pd.to_numeric(trainX[item],errors='coerce')

Handling Categorical Variable: Label encoder 

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [16]:
transformed = trainX.drop(columns=['Tenure','antiguedad','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']).apply(LabelEncoder().fit_transform)

In [17]:
#trainX.columns

Index(['ind_empleado', 'sexo', 'ind_nuevo', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'indfall', 'tipodom', 'ind_actividad_cliente', 'segmento', 'pais_residencia', 'canal_entrada', 'Tenure', 'age', 'renta', 'antiguedad'], dtype='object')

In [18]:
trainX = trainX[['Tenure','antiguedad','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']]

In [19]:
transformed['index'] = transformed.index
trainX['index'] = trainX.index


processed_trainX = pd.merge(transformed,trainX, on='index')

Processing on Test Data

In [49]:
test = pd.read_csv('test_ver2.csv')

In [50]:
test.shape

(929615, 24)

In [60]:
test['indrel_1mes'].fillna(-99,inplace=True)
test['renta'].fillna(101850,inplace=True)
test['ind_nuevo'].fillna(-99,inplace=True)
test['tipodom'].fillna(-99,inplace=True)
test['ind_actividad_cliente'].fillna(-99,inplace=True)
test['age'].fillna(40,inplace=True)
test['indrel'].fillna(-99,inplace=True)

test['tiprel_1mes'].fillna('missing',inplace=True)
test['ind_empleado'].fillna('missing',inplace=True)
test['pais_residencia'].fillna('missing',inplace=True)
test['sexo'].fillna('missing',inplace=True)
test['tiprel_1mes'].fillna('missing',inplace=True)
test['conyuemp'].fillna('missing',inplace=True)
test['segmento'].fillna('missing',inplace=True)
test['canal_entrada'].fillna('missing',inplace=True)
test['indresi'].fillna('missing',inplace=True)
test['indext'].fillna('missing',inplace=True)
test['indfall'].fillna('missing',inplace=True)

test['fecha_dato'] = pd.to_datetime(test['fecha_dato'])
test['fecha_alta'] = pd.to_datetime(test['fecha_alta'])
test['Tenure'] = test['fecha_dato']-test['fecha_alta']


testX = test[cat_cols]

In [62]:
for item in ['Tenure','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']:
    testX[item] = pd.to_numeric(testX[item],errors='coerce')
    
transformed_test = testX.drop(columns=['Tenure','antiguedad','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']).apply(LabelEncoder().fit_transform)

testX_ = testX[['Tenure','antiguedad','age', 'renta','ind_nuevo','tipodom','ind_actividad_cliente','indrel_1mes']]

transformed_test['index'] = transformed_test.index
testX_['index'] = testX_.index


processed_testX = pd.merge(transformed_test,testX_, on='index')

In [66]:
processed_testX.shape

(929615, 20)

### Model 1: 

In [43]:
#processed_trainX

In [44]:
trainX = np.array(processed_trainX)
trainY = np.array(trainY)

In [45]:
classif = OneVsRestClassifier(estimator=LogisticRegression()).fit(trainX, trainY)


Predictions on test set

In [68]:
processed_testX.fillna(0,inplace=True)

In [69]:
predictions = classif.predict()

In [74]:
predictions_targets = pd.DataFrame(predictions,columns=target_cols)

In [76]:
#predictions_targets

In [90]:
#predictions_targets

In [85]:
predictions_targets1 = predictions_targets.replace(1, pd.Series(predictions_targets.columns, predictions_targets.columns))

In [86]:
predictions_targets1

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929610,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
929611,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
929612,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
929613,0,0,ind_cco_fin_ult1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
preds = []
for i,row in predictions_targets1.iterrows():
    #print(row.values)
    temp = ''
    for item in row.values:
        if type(item) is str:
            temp = temp + item +' '
    preds.append(temp)

In [89]:
predictions = pd.DataFrame({'ncodpers': test['ncodpers'], 'segmento': test['segmento'],'added_products': preds})

In [46]:
predictions.to_csv('Submission.csv',index = False)